## Загрузка и обзор данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})


In [2]:
# чтобы выполнить SQL-запрос, используем Pandas
def out(sql_code):
    query = sql_code
    con=engine.connect()
    return pd.io.sql.read_sql(sql=text(query), con = con)

In [3]:
books='''
        SELECT * 
        FROM books 
        LIMIT 5
      '''
out(books)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
authors='''
        SELECT * 
        FROM authors 
        LIMIT 5
        '''
out(authors)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
publishers='''
            SELECT * 
            FROM publishers 
            LIMIT 5
           '''
out(publishers)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
ratings='''
        SELECT * 
        FROM ratings 
        LIMIT 5
        '''
out(ratings)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
reviews='''
        SELECT * 
        FROM reviews 
        LIMIT 5
        '''
out(reviews)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Анализ данных

### Количество книг вышедших после 1 января 2000 года

In [19]:
books = '''
        SELECT COUNT(*)
        FROM books 
        WHERE publication_date > '2000-01-01';
        '''
out(books)

,count
0,819


819 книг вышло после 1 января 2000 года.

### Количество обзоров и средня оценка для каждой книги 

In [18]:
reviews_books = ''' 
                WITH reviews AS
                (SELECT b.book_id,
                b.title,
                COUNT(r.review_id) AS reviews_count
                FROM books b
                LEFT JOIN reviews r on b.book_id = r.book_id
                GROUP BY b.book_id), 
                average AS
                (SELECT b.book_id, 
                AVG(rating) AS average_rating
                FROM books b
                LEFT JOIN ratings rt on b.book_id = rt.book_id
                GROUP BY b.book_id)
                SELECT rev.book_id, rev.title, rev.reviews_count,
                ave.average_rating
                FROM reviews rev
                LEFT JOIN average ave ON rev.book_id = ave.book_id
                ORDER BY reviews_count DESC;
                '''
out(reviews_books)

,book_id,title,reviews_count,average_rating
0,948,Twilight (Twilight #1),7,3.662500
1,963,Water for Elephants,6,3.977273
2,734,The Glass Castle,6,4.206897
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,808,The Natural Way to Draw,0,3.000000
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,221,Essential Tales and Poems,0,4.000000


Больще всего обзоров у "Twilight" - 7, но при этом низкий рейтинг, всего 3.66.
Далее идут "Water for Elephants	" с 6 обзорами и рейтингом 3.97, "The Glass Castle" с 6 обзорами и рейтингом 4.20 и "Harry Potter and the Prisoner of Azkaban" с рейтингом 4.41.

### Издательство, которое выпустило наибольшее число книг толще 50 страниц — исключаем из анализа брошюры

In [ ]:
publisher = '''
            SELECT p.publisher_id,
            p.publisher,
            COUNT(b.book_id) AS book_count
            FROM books b
            JOIN publishers p ON b.publisher_id = p.publisher_id
            WHERE b.num_pages > 50
            GROUP BY p.publisher_id,
            p.publisher
            ORDER BY book_count DESC
            LIMIT 1;
            '''
out(publisher)

Больше всего книг толще 50 страниц выпустило издательство "Penguin Books". Количество выпущенных книг - 42.

### Автор с самой высокой средней оценкой книг — учитывая только книги с 50 и более оценками

In [ ]:
author = '''
         SELECT a.author_id,
         a.author,
         AVG(r.rating) AS average_rating,
         COUNT(r.rating_id) AS rating_count
         FROM authors a
         JOIN books b ON a.author_id = b.author_id
         JOIN 
         (SELECT book_id,
         COUNT(rating_id) AS rating_count
         FROM ratings
         GROUP BY book_id
         HAVING COUNT(rating_id) >= 50
         ) AS subquery ON b.book_id = subquery.book_id
         JOIN ratings r ON b.book_id = r.book_id 
         GROUP BY a.author_id,
         a.author
         ORDER BY average_rating DESC
         LIMIT 1;
         '''
out(author)

Учитывая только те книги, которые имеют более 50 оценок, самая высокая средняя оценка книг у автора "J.K. Rowling/Mary GrandPré" - 4.28. Этот автор имеет 310 оценок.

### Среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [ ]:
user_reviews = '''
               SELECT AVG(review_count) AS average_review_count
               FROM 
               (SELECT r.username,
               COUNT(r.review_id) AS review_count
               FROM reviews r
               JOIN (SELECT username
               FROM ratings
               GROUP BY username
               HAVING COUNT(rating_id) > 48) AS subquery ON r.username = subquery.username
               GROUP BY r.username) AS user_reviews;
               '''
out(user_reviews)

Учитывая только тех пользователей, которые поставили более 48 оценок, среднее количество написанных обзоров составляет 24.

## Вывод

На основе предоставленной информации можно сделать следующие выводы и рекомендации:

1.	Возросшее количество выпущенных книг после 1 января 2000 года (821 книга) может указывать на активность издательств и повышенный интерес к чтению книг. Это может быть положительным сигналом для компании, которая приобрела сервис для чтения книг по подписке.

2.	Анализ количества обзоров и средней оценки для каждой книги может помочь определить популярность и качество книг в базе данных. Оценка качества книг и отзывов пользователей может быть важным фактором для формулирования ценностного предложения нового продукта.

3.	Издательство Penguin Books выпустило наибольшее количество книг, что указывает на его значительную роль на рынке и потенциальную популярность среди читателей. Рекомендация состоит в том, чтобы обратить внимание на книги, выпущенные издательством Penguin Books, и оценить их качество и подходят ли они под вашу целевую аудиторию.

4.	Автор J.K. Rowling имеет самую высокую среднюю оценку книг среди авторов с 50 и более оценками. Это указывает на популярность и высокую оценку ее произведений среди читателей. Стоит обратить внимание на книги J.K. Rowling и рассмотреть возможность включения их в подписку на чтение книг.

5.	Пользователи, которые оставили более 48 оценок, в среднем написали 24 обзора. Это говорит о том, что эти пользователи активно взаимодействуют с книгами, оценивают их и оставляют обзоры. Нужно обратить внимание на обзоры этих пользователей, так как они могут быть полезными для привлечения новых пользователей и формирования репутации сервиса.
